# Finding Similar Items: Textually Similar Documents


## Task
The task of this lab was to implement the stages of finding textually similar documents based on Jaccard similarity using the shingling, minhashing, and locality-sensitive hashing (LSH) techniques and corresponding algorithms. Theory can be found in the docs directory. 

## How to Run 
Run the code by sequentially executing each block of the code. Hyperparameters can be set for part 1, 2 and 3 under each parts respective section *Hyperparameters*. 

In [678]:
## !!! typical mistake: comparing signatures is given as probability. Using jaccard to compute this will result ina division by 200 which does not make sense! 
import pandas as pd
import numpy as np
import hashlib
from OrderedSet import OrderedSet

In [679]:
## For comparing the full files to each other, use this: 

import os

path = "../data/OpinRankDataset/hotels/small_sample/"
raw_documents = []

for root, dirs, files in os.walk(path):  
    for filename in files:
        raw_doc = ""
        with open(path + filename, encoding='utf-8', errors='replace') as f:
            for line in f.readlines():
                raw_doc += line
        raw_documents.append(raw_doc)

In [680]:
## For shorter runtime, just compare different reviews with each other. Run this: 


#with open("../data/OpinRankDataset/hotels/small_sample/usa_san francisco_abigail_hotel", encoding='utf-8', errors='replace') as f: 
#    raw_documents = f.readlines()
    

In [681]:
## Number of documents
print(len(raw_documents))

9


In [694]:
class Document: 
    def __init__(this, text, n_buckets, shingle_length, filename):
        this.filename = filename
        this.text = text
        this.length = len(text)
        this.n_buckets = n_buckets
        this.shingle_length = shingle_length
    
    def createShingles(this): 
        this.shingles = OrderedSet()
        
        ## number of shingles = length/k
        n = int(this.length/this.shingle_length)
        
        ## slice string
        for i in range(n-1):
            this.shingles.add(this.text[slice(this.shingle_length * i, this.shingle_length * (i+1))])
    
    def hashShingles(this):
        this.hashedShingles = OrderedSet()
        
        for shingle in this.shingles: 
            this.hashedShingles.add(this.hashStr(shingle))
    
    def hashStr(this, string):
        ## Hashes a string to an integer 0 <= i < n_buckets
        return abs(hash(string)) % (this.n_buckets)
    
    def jaccardSimilarity(this, other): 
        ## Compares two docs based on the jaccard similarity of the hashed shingles. 
        intersection = (this.hashedShingles & other.hashedShingles)
        union = (this.hashedShingles | other.hashedShingles)
        return (len(intersection)/len(union))
    
    def createSignature(this, h):
        ## builds a minHash signature (in the form of a vector or a set) of a given length n from a given set of integers (a set of hashed shingles).
        this.signature = np.ones(h.n_hashes) * np.inf ## the signature has length k, corresponding to each hash
        for i in range(h.n_hashes):
            for e in this.shingles:
                this.signature[i] = np.minimum(this.signature[i], h.hash(i, e)) # keep the smallest hash value of each hash function only
        
    def signatureSimilarity(this, other):
        ## estimates similarity of two integer vectors – minhash signatures – as a fraction of components, in which they agree.
        agree = 0
        
        for i in range(this.signature.size): 
            if (this.signature[i] == other.signature[i]): 
                agree += 1
                
        similarity = agree/this.signature.size
        return similarity
        

In [695]:
class Hasher: 
    def __init__(this, n_hashes, n_buckets): 
        ## A class that contains k random hash functions
        this.n_hashes = n_hashes
        this.n_buckets = n_buckets
        
        
    def hash(this, i, string): 
        ## returns hash function i of x: h_i(x)
        a = this.a_values[i]
        b = this.b_values[i]
        x = abs(hash(string))
        return (a*x + b) % this.n_buckets
    
    def generateRandomHashFunctions(this):
        # Each hash function is a function of random parameters a, b and c. 
        # create a and b values: 0 < random_interval_size
        this.a_values = (np.random.rand(this.n_hashes) * this.n_buckets).astype(int)
        this.b_values = (np.random.rand(this.n_hashes) * this.n_buckets).astype(int)
        
    def getHashMatrix(this, shingles):
        # Creates a hash matrix based on an input array of shingles
        # M(i, j) = h_i(e_j)
        this.hash_matrix = np.zeros((this.n_hashes, shingles.size))
        for i in range(this.n_hashes): # for each hash function
            for e in shingles:
                this.hash_matrix[i, j] = this.hash(i, e)
                
    

In [696]:
class LSH: 
    def __init__(this, n_rows, n_bands, t, n_buckets):
        this.n_rows = n_rows # rows per band
        this.n_bands = n_bands
        this.t = t
        this.n_buckets = n_buckets
        this.hash_weights = (np.random.rand(this.n_rows) * n_buckets).astype(int)
        
    def getCandidatePairs(this, documents):
        # takes a list of documents, returns a set of similar pairs based on LSH of their signatures. 
        bucket_doc_mat = np.zeros((this.n_buckets, len(documents)))
        this.candidate_pairs = set()
        for docindex, d in enumerate(documents):
        # for each doc
            for b in range(n_bands):
            # for each band 
                band = d.signature[b * n_rows: (b+1) * n_rows]
                hb = this.hashBand(band)
                # hash band
                # put 1 in bucket-doc matrix
                bucket_doc_mat[hb, docindex] = 1

        for bucket in bucket_doc_mat:
        # for each bucket
            doc_indices = np.nonzero(bucket)[0] # returns indices of all documents in the bucket
            # add each pair to set
            for d1 in doc_indices: # slow and stupid solution but concept works
                for d2 in doc_indices: 
                    if (d1 < d2): ## make sure we just add each pair once..
                        this.candidate_pairs.add((d1, d2))

        return this.candidate_pairs
    
    def getSimilarPairs(this, documents):
        ## Returns all similar pairs, based on candidate pairs and signature similarity >= t
        this.similar_pairs = set()
        for (d1, d2) in this.candidate_pairs: 
            if (documents[d1].signatureSimilarity(documents[d2]) >= this.t):
                this.similar_pairs.add((d1, d2, documents[d1].signatureSimilarity(documents[d2])))

        return this.similar_pairs
    
    def hashBand(this, band): 
        # takes the band (an array of integers) and hashes it to a
        band_weight_sum = 0
        for i, row in enumerate(band):
            band_weight_sum += (this.hash_weights[i] * row) 
            
        hashed_band = int(band_weight_sum % this.n_buckets)
        return int(hashed_band)
    
    

# 1. Jaccard Similarity of sets 

- Create shingle representation
- Create hashed version of the representation. 
- Compute Jaccard Similarity of documents based on sets of hashed shingles 


## Hyperparameters

#### n_buckets
The number of hash buckets (or the number of different possible output values for the hash function). A high n_buckets will yield low similarity measures, because more possible hash values means fewer shingles will be hashed to the same buckets.

#### shingle_length
The number of characters in each shingle. Large value will yield low similarity measures, as there will be more combinations of shingles. 

In [697]:
n_buckets = 2 ** 11 # number of hash buckets. This strongly affects the measure of similarity
shingle_length = 10

### Create documents. add to an array Documents


In [698]:
documents = []
for i, d in enumerate(raw_documents):
    documents.append(Document(d, n_buckets, n_shingles, filename = files[i]))


### Example of document: 

In [699]:
documents[0].text

"Oct 30 2009 \tOK value though shabby; good position.\tI extended a one night stay to three. The staff were basically helpful when asked. My room was clean, comfortable, and daily serviced. I think the position is good for a budget hotel, near the Golden Gate theatre, right by Asian Arts Museum and Civic Centre. It's an old and architecturally atmospheric hotel.Negatives: it is very badly neglected. Things that might put you off are the need usually to ring the outside doorbell and wait to get someone to the unattended reception, unreliable breakfast availability (nil when I was there but I can see from other reviews that it can appear!), a very unreliable (but rather beautiful) classic Otis elevator, some very dilapidated corridors on the way to your room when the elevator is kaput, shabby though formerly elegant decor, homeless on nearby streets, though the street immediately outside was clear and felt safe and I didn't see any drug taking or pushing and I was never bothered anywhere

In [700]:
for d in documents: 
    d.createShingles()
    d.hashShingles() 
    
print("Jaccard Similarity of doc 1 and doc 2 is {}".format(documents[1].jaccardSimilarity(documents[2])))

Jaccard Similarity of doc 1 and doc 2 is 0.5149436550710436


In [702]:
## Example of shingles 
documents[0].shingles

OrderedSet(['Oct 30 200', '9 \tOK valu', 'e though s', 'habby; goo', 'd position', '.\tI extend', 'ed a one n', 'ight stay ', 'to three. ', 'The staff ', 'were basic', 'ally helpf', 'ul when as', 'ked. My ro', 'om was cle', 'an, comfor', 'table, and', ' daily ser', 'viced. I t', 'hink the p', 'osition is', ' good for ', 'a budget h', 'otel, near', ' the Golde', 'n Gate the', 'atre, righ', 't by Asian', ' Arts Muse', 'um and Civ', 'ic Centre.', " It's an o", 'ld and arc', 'hitectural', 'ly atmosph', 'eric hotel', '.Negatives', ': it is ve', 'ry badly n', 'eglected. ', 'Things tha', 't might pu', 't you off ', 'are the ne', 'ed usually', ' to ring t', 'he outside', ' doorbell ', 'and wait t', 'o get some', 'one to the', ' unattende', 'd receptio', 'n, unrelia', 'ble breakf', 'ast availa', 'bility (ni', 'l when I w', 'as there b', 'ut I can s', 'ee from ot', 'her review', 's that it ', 'can appear', '!), a very', ' unreliabl', 'e (but rat', 'her beauti', 'ful) class', 'ic Otis el', 'evato

In [703]:
## Example of hashed shingles 
documents[0].hashedShingles

OrderedSet([1513, 343, 619, 1515, 592, 996, 398, 291, 1530, 627, 1406, 1742, 783, 1828, 1595, 1110, 151, 359, 1604, 123, 404, 491, 96, 1063, 493, 907, 1930, 1316, 1233, 550, 286, 1942, 2028, 1066, 16, 664, 218, 1830, 441, 1127, 2043, 556, 276, 1725, 242, 6, 543, 1970, 235, 1179, 2011, 1387, 1107, 527, 95, 1663, 587, 23, 382, 1809, 1240, 1022, 137, 454, 1153, 1714, 225, 68, 626, 1045, 98, 1769, 1740, 502, 1478, 1852, 1884, 1337, 963, 855, 724, 1398, 142, 1099, 1001, 1255, 315, 439, 761, 547, 131, 569, 282, 1191, 1304, 1493, 1929, 1369, 55, 341, 1232, 1555, 1958, 320, 795, 1329, 1248, 125, 1273, 1838, 70, 47, 1332, 396, 1627, 1697, 1995, 1524, 1287, 737, 1669, 1853, 661, 1650, 1825, 485, 293, 1479, 1201, 734, 392, 2038, 1813, 940, 1906, 1148, 1778, 2022, 1869, 1891, 1252, 1112, 1785, 1400, 1020, 1864, 160, 1560, 331, 457, 1926, 807, 278, 1176, 1141, 953, 1470, 1777, 1163, 295, 1253, 787, 808, 175, 1097, 997, 364, 879, 2041, 794, 260, 108, 579, 1846, 1093, 1541, 954, 1557, 1269, 348, 1436

### Example: compute similarities for 10 documents.

In [704]:
# Compute jaccard of some documents and store in dataframe
n_docs = 9
similarity_threshold = 0.5
similarities = np.zeros((n_docs, n_docs))
for i in range(n_docs): 
    for j in range(n_docs): 
        similarities[i, j] = documents[i].jaccardSimilarity(documents[j])
        
simDF = pd.DataFrame(similarities)
simDF.head(n_docs)

,0,1,2,3,4,5,6,7,8
0,1.000000,0.771526,0.450411,0.319588,0.550556,0.524880,0.489707,0.634083,0.422149
1,0.771526,1.000000,0.514944,0.346889,0.651493,0.613614,0.566176,0.747920,0.489706
2,0.450411,0.514944,1.000000,0.273314,0.408451,0.375000,0.370988,0.462275,0.341780
3,0.319588,0.346889,0.273314,1.000000,0.304375,0.291123,0.276683,0.307425,0.250545
4,0.550556,0.651493,0.408451,0.304375,1.000000,0.469791,0.439333,0.538749,0.387116
5,0.524880,0.613614,0.375000,0.291123,0.469791,1.000000,0.412587,0.514332,0.368739
6,0.489707,0.566176,0.370988,0.276683,0.439333,0.412587,1.000000,0.496674,0.355000
7,0.634083,0.747920,0.462275,0.307425,0.538749,0.514332,0.496674,1.000000,0.420436
8,0.422149,0.489706,0.341780,0.250545,0.387116,0.368739,0.355000,0.420436,1.000000


## Jaccard Similarity results 
### Find similar docs

In [705]:
similar = []
for i in range(n_docs): 
    for j in range(n_docs): 
        if(similarities[i, j] > similarity_threshold):
            if (i < j): ## This is in order to just include each pair once.. 
                print("Documents {} and {} are similar, similarity {}".format(i, j, round(similarities[i, j], 4)))
                similar.append((i, j))

Documents 0 and 1 are similar, similarity 0.7715
Documents 0 and 4 are similar, similarity 0.5506
Documents 0 and 5 are similar, similarity 0.5249
Documents 0 and 7 are similar, similarity 0.6341
Documents 1 and 2 are similar, similarity 0.5149
Documents 1 and 4 are similar, similarity 0.6515
Documents 1 and 5 are similar, similarity 0.6136
Documents 1 and 6 are similar, similarity 0.5662
Documents 1 and 7 are similar, similarity 0.7479
Documents 4 and 7 are similar, similarity 0.5387
Documents 5 and 7 are similar, similarity 0.5143


In [706]:
## Show first part of two similar'docs
print(documents[similar[0][0]].text[0:500])
print("\n")
print(documents[similar[0][1]].text[0:500])

Oct 30 2009 	OK value though shabby; good position.	I extended a one night stay to three. The staff were basically helpful when asked. My room was clean, comfortable, and daily serviced. I think the position is good for a budget hotel, near the Golden Gate theatre, right by Asian Arts Museum and Civic Centre. It's an old and architecturally atmospheric hotel.Negatives: it is very badly neglected. Things that might put you off are the need usually to ring the outside doorbell and wait to get some


Oct 28 2009 	Good value	Nice little hotel in a great location. Not 5* but then you're not paying 5* rates. Clean, friendly staff and an easy 5 minute walk to Union Square.	
Oct 25 2009 	Good location but cramped and unclean	We stayed in the Adante for two nights when we arrived in SF and again for one night the following week before we left. The hotel is ideally situated only a short walk from Union Square and the front desk staff were helpful. We were allowed to check-in early and were even 

# 2. Min-Hashing
- Create a set of k random hash function
- Get the signature of each document by: 
    - for each shingle e in doc
        - save the index i of the hash function producing smallest h(e)
        
        
## Hyperparameters

#### n_buckets
The number of hash buckets (or the number of different possible output values for the hash function). A high n_buckets will yield low similarity measures, because more possible hash values means fewer shingles will be hashed to the same buckets.

#### n_hashes 
The number of hash functions used to create the min hashing signature. 



In [707]:
n_hashes = 100
n_buckets = 2 ** 11 # number of hash buckets. This strongly affects the measure of similarity


In [708]:
## Create a set of k random hash functions (permutations). 
h = Hasher(n_hashes, n_buckets)
h.generateRandomHashFunctions()

## Create signatures for each set
for count, d in enumerate(documents):
    d.createSignature(h)
    print("Processed doc {}".format(count))


/Users/philipclaesson/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in long_scalars
  del sys.path[0]


Processed doc 0
Processed doc 1
Processed doc 2
Processed doc 3
Processed doc 4
Processed doc 5
Processed doc 6
Processed doc 7
Processed doc 8


In [709]:
# Example of document signature
documents[0].signature

array([  0.,   3.,   0.,   0.,   0.,   1.,   3.,   4.,   0.,   0.,  11.,
         2.,   1.,   0.,   0.,   1.,   0.,   1.,   0.,   1.,   0.,   1.,
         0.,  28.,   1.,   0.,   2.,   0.,  12.,   0.,   0.,   1.,   6.,
         0.,   0.,   0.,   0.,   1.,   7.,   0.,   1.,   1.,   3.,   0.,
         1.,   1.,   0.,   0.,   1.,   1.,   0.,   0.,   1.,   1.,   0.,
         1.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,
         2.,   1.,   0.,   0.,   0.,   1.,   1.,   1.,   0.,   0.,   1.,
        29.,   0.,   0.,   0.,   0.,   1.,   0.,   1.,   1.,   2.,   1.,
         0.,   0.,   0.,   2.,   1.,   0.,   0.,   0.,   1.,   1.,   2.,
         1.])

### Example: compute similarities for 10 documents.

In [710]:
# Compute minhash signature similarity of some documents and store in dataframe
n_docs = 9
similarity_threshold = 0.7
similarities = np.zeros((n_docs, n_docs))
for i in range(n_docs): 
    for j in range(n_docs): 
        similarities[i, j] = documents[i].signatureSimilarity(documents[j])
        
simDF = pd.DataFrame(similarities)
simDF.head(n_docs)

,0,1,2,3,4,5,6,7,8
0,1.00,0.86,0.62,0.43,0.74,0.68,0.63,0.79,0.64
1,0.86,1.00,0.64,0.47,0.81,0.73,0.69,0.90,0.71
2,0.62,0.64,1.00,0.51,0.57,0.60,0.56,0.60,0.54
3,0.43,0.47,0.51,1.00,0.50,0.42,0.36,0.45,0.37
4,0.74,0.81,0.57,0.50,1.00,0.66,0.66,0.77,0.61
5,0.68,0.73,0.60,0.42,0.66,1.00,0.50,0.70,0.60
6,0.63,0.69,0.56,0.36,0.66,0.50,1.00,0.68,0.57
7,0.79,0.90,0.60,0.45,0.77,0.70,0.68,1.00,0.67
8,0.64,0.71,0.54,0.37,0.61,0.60,0.57,0.67,1.00


## Min-hashing results
### Find similar docs

In [711]:
similar = []
for i in range(n_docs): 
    for j in range(n_docs): 
        if(similarities[i, j] > similarity_threshold):
            if (i < j): ## This is in order to just include each pair once.. 
                print("Documents {} and {} are similar, similarity {}".format(i, j, round(similarities[i, j], 4)))
                similar.append((i, j))
    
    

Documents 0 and 1 are similar, similarity 0.86
Documents 0 and 4 are similar, similarity 0.74
Documents 0 and 7 are similar, similarity 0.79
Documents 1 and 4 are similar, similarity 0.81
Documents 1 and 5 are similar, similarity 0.73
Documents 1 and 7 are similar, similarity 0.9
Documents 1 and 8 are similar, similarity 0.71
Documents 4 and 7 are similar, similarity 0.77


# 3. Locality Sensitive Hashing 
- Hash bands of signatures of each document into buckets 
- For each bucket, add all docs in the same bucket to a a set of Candidate Pairs
- Filter out candidate pairs which are less similar than t (false positives) to get the Similar Pairs

## Hyperparameters

### n_bands and n_rows
The number of bands, and number of rows in each band. b * r should be equal to the number of integers in the signature of a document, n. 

### t
The threshold of similarity to filter out not false positives from candidate pairs. 

### n_lsh_buckets
The number of buckets in the hash function which hashes the bands of the signatures. Should be as large as possible, but increases the runtime! 

In [712]:
## Hyperparameters
n_bands = 20
n_rows = 5 # rows per band
t = 0.55 # (1/n_bands) ** (1/n_rows) = 0.549
n_lsh_buckets = 2 ** 20

In [713]:
lsh = LSH(n_bands = n_bands, n_rows = n_rows, t = t, n_buckets = n_lsh_buckets)

candidate_pairs = lsh.getCandidatePairs(documents)

similar_pairs = lsh.getSimilarPairs(documents) 

In [714]:
print("Out of {} documents, {} candidate pairs were found. Out of these, {} had a similarity of at least {}. "
      .format(len(documents), len(candidate_pairs), len(similar_pairs), t))



Out of 9 documents, 28 candidate pairs were found. Out of these, 25 had a similarity of at least 0.55. 


## LSH Result
### Find similar docs

In [650]:
for d1, d2, sim in (similar_pairs): 
    if (sim > similarity_threshold):
        print("Documents {} and {} are similar, similarity {}".format(d1, d2, round(sim, 4)))

    

Documents 4 and 7 are similar, similarity 0.78
Documents 6 and 7 are similar, similarity 0.72
Documents 0 and 5 are similar, similarity 0.72
Documents 1 and 4 are similar, similarity 0.89
Documents 1 and 7 are similar, similarity 0.87
Documents 4 and 5 are similar, similarity 0.72
Documents 1 and 2 are similar, similarity 0.73
Documents 0 and 6 are similar, similarity 0.74
Documents 1 and 8 are similar, similarity 0.71
Documents 2 and 6 are similar, similarity 0.71
Documents 1 and 6 are similar, similarity 0.78
Documents 4 and 6 are similar, similarity 0.71
Documents 0 and 7 are similar, similarity 0.8
Documents 0 and 4 are similar, similarity 0.77
Documents 0 and 1 are similar, similarity 0.87
Documents 1 and 5 are similar, similarity 0.74


In [716]:
print("Doc 0 and 1 are considered among the most equal, they correspond to the files '{} and '{}' which can be found in ".format(documents[0].filename, documents[1].filename))

Doc 0 and 1 are considered among the most equal, they correspond to the files 'usa_san francisco_abigail_hotel and 'usa_san francisco_adante_hotel'
